<a href="https://colab.research.google.com/github/LeeSeungwon89/Kaggle_Dacon_Practice/blob/main/4.%20Porto_Seguro%E2%80%99s_Safe_Driver_Prediction_improve_model_performance2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [2]:
ls -1ha kaggle.json

kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

# 참가한 대회 리스트를 확인합니다.
# !kaggle competitions list

In [4]:
!kaggle competitions download -c porto-seguro-safe-driver-prediction

 88% 67.0M/76.5M [00:00<00:00, 79.2MB/s]
100% 76.5M/76.5M [00:00<00:00, 89.0MB/s]


In [5]:
!ls

kaggle.json  porto-seguro-safe-driver-prediction.zip  sample_data


In [6]:
!unzip porto-seguro-safe-driver-prediction.zip

Archive:  porto-seguro-safe-driver-prediction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# **1. 피처 엔지니어링**

이번 성능 개선 챕터에서는 파생 피처를 생성해 성능을 측정해 보겠습니다. 하이퍼파라미터 튜닝은 다음 성능 개선 챕터에서 수행하겠습니다.

## **1.1. 데이터 준비**

데이터를 준비하겠습니다.

In [7]:
import numpy as np
import pandas as pd

# 최대 행렬 수를 설정합니다.
pd.set_option('display.max_column', 80)
pd.set_option('display.max_rows', 80)

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [8]:
all_data = pd.concat([train, test], ignore_index=True) # 인덱스를 순차형으로 지정합니다.
all_data

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0.0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0.0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0.0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0.0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0.0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488023,1488022,NaN,0,1,6,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0.5,0.3,1.048809,0,1,-1,8,0,14,0,1,0,1,18,2,0.565685,1.528774,0.565685,3.605551,0.3,0.4,0.9,1,1,7,3,10,3,10,4,2,3,4,0,1,0,0,1,0
1488024,1488023,NaN,5,3,5,1,0,0,0,1,0,0,0,0,0,0,11,1,0,0,0.7,1.0,1.246495,8,1,-1,0,0,10,1,0,0,1,83,2,0.424264,1.040567,0.418569,3.741657,0.3,0.2,0.6,1,3,7,3,10,4,9,6,2,2,11,0,0,1,1,0,0
1488025,1488024,NaN,0,1,5,0,0,1,0,0,0,0,0,0,0,0,5,0,0,1,0.4,0.0,0.609303,11,1,-1,0,-1,11,1,0,1,1,17,2,0.374166,0.968992,0.390384,3.741657,0.3,0.3,0.9,2,1,10,4,8,2,8,5,2,2,11,0,1,1,0,0,0
1488026,1488025,NaN,6,1,5,1,0,0,0,0,1,0,0,0,0,0,13,1,0,0,0.6,0.6,0.920937,10,0,-1,0,1,10,1,1,0,1,28,3,0.387298,0.725125,0.378153,2.645751,0.1,0.1,0.3,1,1,9,2,10,1,8,1,1,2,7,1,1,0,0,0,0


In [9]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1488028 entries, 0 to 1488027
Data columns (total 59 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   id              1488028 non-null  int64  
 1   target          595212 non-null   float64
 2   ps_ind_01       1488028 non-null  int64  
 3   ps_ind_02_cat   1488028 non-null  int64  
 4   ps_ind_03       1488028 non-null  int64  
 5   ps_ind_04_cat   1488028 non-null  int64  
 6   ps_ind_05_cat   1488028 non-null  int64  
 7   ps_ind_06_bin   1488028 non-null  int64  
 8   ps_ind_07_bin   1488028 non-null  int64  
 9   ps_ind_08_bin   1488028 non-null  int64  
 10  ps_ind_09_bin   1488028 non-null  int64  
 11  ps_ind_10_bin   1488028 non-null  int64  
 12  ps_ind_11_bin   1488028 non-null  int64  
 13  ps_ind_12_bin   1488028 non-null  int64  
 14  ps_ind_13_bin   1488028 non-null  int64  
 15  ps_ind_14       1488028 non-null  int64  
 16  ps_ind_15       1488028 non-null  in

'id'와 'target' 피처를 제거하겠습니다.

In [10]:
all_data.drop(['id', 'target'], axis=1, inplace=True)

## **1.2. 파생 피처 생성(작업 중)**

파생 피처를 생성해 보겠습니다. 불필요한 피처를 제거하는 작업은 파생 피처를 생성한 후 수행할 것입니다. 본래 존재하는 피처를 최대한 활용하여 파생 피처를 생성해야 하기 때문입니다.

### **1.2.1. 결측치 개수 피처**

결측치 개수를 가진 피처를 생성해 보겠습니다. 각 명목형 피처에 결측치는 -1로 처리되어 있습니다. 다시 말해 각 데이터(=행, 레코드, 튜플 등)가 가진 -1의 개수를 합산한 결과를 피처로 생성하는 것입니다.

먼저 -1이 포함된 피처를 확인해 보겠습니다.

In [11]:
feature_list = [column for column in all_data.columns if all_data[all_data == -1][column].any()]
print(feature_list)

['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_reg_03', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_05_cat', 'ps_car_07_cat', 'ps_car_09_cat', 'ps_car_11', 'ps_car_12', 'ps_car_14']


명목형 피처가 아닌 피처도 -1이 포함되어 있지만 큰 영향은 없을 것으로 판단합니다. 그대로 파생 피처를 생성하겠습니다.

In [12]:
# `axis=1`로 지정하여 데이터마다 결측치 개수를 산출합니다.
# `axis=0`으로 지정하면 열마다 결측치 개수가 산출됩니다.
all_data['count_missing_data'] = (all_data == -1).sum(axis=1)

In [13]:
all_data['count_missing_data']

0          1
1          2
2          3
3          0
4          2
          ..
1488023    1
1488024    1
1488025    2
1488026    1
1488027    0
Name: count_missing_data, Length: 1488028, dtype: int64

가장 많은 결측치를 가진 데이터를 확인해 보겠습니다.

In [14]:
all_data['count_missing_data'].max()

8

### **1.2.2. 고윳값 개수 피처**



---

작업 중

---



In [15]:
list_category =[]
list_bin = []
list_continuous = []
list_ordinal = []

for column in all_data.columns:
    if 'cat' in column:
        list_category.append(column)
    elif ('bin' in column):
        list_bin.append(column)
    elif all_data[column].dtype == 'float64':
        list_continuous.append(column)
    elif (all_data[column].dtype == 'int64'):
        list_ordinal.append(column)

In [16]:
list_ind = [feature for feature in all_data.columns if 'ind' in feature]
list_ind

['ps_ind_01',
 'ps_ind_02_cat',
 'ps_ind_03',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_10_bin',
 'ps_ind_11_bin',
 'ps_ind_12_bin',
 'ps_ind_13_bin',
 'ps_ind_14',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin']

In [17]:
first_value = True

for feature in list_ind:
    if first_value:
        all_data['ind_values_connected'] = all_data[feature].astype(str)
        first_value = False
    else:
        all_data['ind_values_connected'] += all_data[feature].astype(str)

In [18]:
all_data['ind_values_connected']

0          2251001000000011010
1           117000010000003001
2          5491000100000012100
3           012001000000008100
4           020101000000009100
                  ...         
1488023     016000100000002001
1488024    5351000100000011100
1488025     015001000000005001
1488026    6151000010000013100
1488027    7141000010000012100
Name: ind_values_connected, Length: 1488028, dtype: object

In [19]:
all_data['ind_values_connected'].value_counts()

021001000000007100     2992
012001000000008100     2784
012001000000007100     2568
011001000000007100     2174
020001000000007100     2131
                       ... 
020060001000004010        1
647000010000007100        1
3481200010000012100       1
546120100000004100        1
518000100010013001        1
Name: ind_values_connected, Length: 143769, dtype: int64

In [20]:
list_count_features = []
for feature in list_category + ['ind_values_connected']:
    value_dictionary = all_data[feature].value_counts().to_dict()
    lambda_values_count = lambda x: value_dictionary[x]
    all_data[f'{feature}_values_count'] = all_data[feature].apply(lambda_values_count)
    list_count_features.append(f'{feature}_values_count')

In [21]:
list_count_features

['ps_ind_02_cat_values_count',
 'ps_ind_04_cat_values_count',
 'ps_ind_05_cat_values_count',
 'ps_car_01_cat_values_count',
 'ps_car_02_cat_values_count',
 'ps_car_03_cat_values_count',
 'ps_car_04_cat_values_count',
 'ps_car_05_cat_values_count',
 'ps_car_06_cat_values_count',
 'ps_car_07_cat_values_count',
 'ps_car_08_cat_values_count',
 'ps_car_09_cat_values_count',
 'ps_car_10_cat_values_count',
 'ps_car_11_cat_values_count',
 'ind_values_connected_values_count']



---

작업 중

---



## **1.2. 불필요한 피처 제거**

반드시 제거해야 할 피처뿐만 아니라 제거 여부를 확정할 수 없는 피처까지 모두 제거하겠습니다.

In [ ]:
drop_all = ['ps_car_10_cat', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_13_bin',
            'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
            'ps_calc_19_bin', 'ps_calc_20_bin', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03',
            'ps_ind_14', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06', 'ps_calc_07',
            'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11', 'ps_calc_12',
            'ps_calc_13', 'ps_calc_14']
all_data.drop(drop_all, axis=1, inplace=True)

In [ ]:
all_data.head(3)

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_12_bin,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,2,2,5,1,0,0,1,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,12,2,0.400000,0.883679,0.370810,3.605551
1,1,1,7,0,0,0,0,1,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,19,3,0.316228,0.618817,0.388716,2.449490
2,5,4,9,1,0,0,0,1,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,60,1,0.316228,0.641586,0.347275,3.316625


## **1.3. 명목형 피처에 원-핫 인코딩**

명목형 피처에 원-핫 인코딩을 수행하겠습니다. 먼저 각 피처명을 리스트에 분류하겠습니다.

In [ ]:
list_category =[]
list_bin = []
list_continuous = []
list_ordinal = []

for column in all_data.columns:
    if 'cat' in column:
        list_category.append(column)
    elif ('bin' in column):
        list_bin.append(column)
    elif all_data[column].dtype == 'float64':
        list_continuous.append(column)
    elif (all_data[column].dtype == 'int64'):
        list_ordinal.append(column)

명목형 피처에 원-핫 인코딩을 수행하여 따로 객체화하겠습니다. 아래에서 전체 데이터와 결합할 예정입니다.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()
category_one_hot_encoded = one_hot_encoder.fit_transform(all_data[list_category])

In [ ]:
category_one_hot_encoded

<1488028x181 sparse matrix of type '<class 'numpy.float64'>'
	with 19344364 stored elements in Compressed Sparse Row format>

전체 데이터에서 명목형 피처를 제거해 두겠습니다.

In [ ]:
all_data.drop(list_category, axis=1, inplace=True)

## **1.4. 순서형 피처에 min-max 정규화**

순서형 피처에 min-max 정규화를 적용하겠습니다.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
all_data[list_ordinal] = min_max_scaler.fit_transform(all_data[list_ordinal])

In [ ]:
all_data[list_ordinal].head(3)

,ps_ind_01,ps_ind_03,ps_ind_15,ps_car_11
0,0.285714,0.454545,0.846154,0.75
1,0.142857,0.636364,0.230769,1.00
2,0.714286,0.818182,0.923077,0.50


## **1.5. 전체 데이터와 매트릭스 결합**

전체 데이터와 원-핫 인코딩 된 명목형 피처 매트릭스를 결합하겠습니다. 

In [ ]:
from scipy import sparse

all_data_sparse_hstack = sparse.hstack([sparse.csr_matrix(all_data),
                                       category_one_hot_encoded], format='csr')

In [ ]:
all_data_sparse_hstack

<1488028x200 sparse matrix of type '<class 'numpy.float64'>'
	with 37707591 stored elements in Compressed Sparse Row format>

# **2. 모델링**

## **2.1. 데이터 준비**

데이터를 준비하겠습니다.

In [ ]:
size_train = len(train)
X_train_sparse = all_data_sparse_hstack[:size_train]
X_test_sparse = all_data_sparse_hstack[size_train:]
y_train_original = train['target'].values

## **2.2. 정규화된 지니계수 산출 함수 선언**

정규화된 지니계수를 계산하는 함수를 선언하겠습니다. 참고로 지니계수는 값이 클수록 불평등함을 의미하고, 정규화된 지니계수는 값이 클수록 좋은 성능을 의미합니다. 

In [ ]:
def normalized_gini_coefficient_2(y_true, y_prediction): # `y_true`는 실젯값, `y_prediction`은 예측값입니다.
    # 실젯값 크기와 예측값 크기를 대조합니다.
    assert y_true.shape == y_prediction.shape

    number_samples = y_true.shape[0] # 데이터 개수를 객체화합니다.
    lolenz_middle = np.linspace(1 / number_samples, 1, number_samples) # 대각선 값을 객체화합니다.

    # 예측값의 지니계수를 산출합니다.
    prediction_order = y_true[y_prediction.argsort()] # `y_prediction` 크기 순서에 맞추어 `y_true` 값을 재정렬합니다.
    lolenz_prediction = np.cumsum(prediction_order) / np.sum(prediction_order) # 로렌츠 곡선을 객체화합니다.
    gini_prediction = np.sum(lolenz_middle - lolenz_prediction)

    # 모든 예측이 정확한 경우의 지니계수를 산출합니다.
    true_order = y_true[y_true.argsort()] # `y_true` 크기 순서에 맞추어 `y_true` 값을 재정렬합니다.
    lolenz_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선을 객체화합니다.
    gini_true = np.sum(lolenz_middle - lolenz_true)

    # 정규화된 지니계수를 산출합니다.
    normalized_gini_coefficient = gini_prediction / gini_true

    return normalized_gini_coefficient

## **2.3. 층화 K-폴드 및 OOF 예측을 위한 변수 준비**

타깃값이 불균형하므로 층화 K 폴드 방식을 채택하겠습니다. 아울러 성능 측정에 OOF prediction 방식을 활용하기 위해 예측 확률을 저장하는 1차원 배열 객체를 생성하겠습니다.

In [ ]:
from sklearn.model_selection import StratifiedKFold

stratified_kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)

# 1차원 배열을 생성합니다.
oof_valid_predictions = np.zeros(X_train_sparse.shape[0])
oof_test_predictions = np.zeros(X_test_sparse.shape[0])

## **2.4. 모델 훈련**

데이터셋 크기가 크기 때문에 훈련 속도가 빠른 LightGBM을 적용하겠습니다. 이번에는 파이썬 래퍼 형식으로 훈련해 보겠습니다.

In [ ]:
# 파이썬 래퍼 형식
import lightgbm as lgb

# 모델을 훈련할 때 검증용으로 지정할 함수를 선언합니다.
def gini_feval(predictions, dtrain):
    labels = dtrain.get_label() # 데이터셋의 타깃값을 반환합니다.
    return 'gini', normalized_gini_coefficient_2(labels, predictions), True
# 함수 끝

parameters = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'force_row_wise': True, # 경고 메시지를 띄우지 않습니다.
    'random_state': 2023
}

for index, (train_index, valid_index) in enumerate(stratified_kfolds.split(X_train_sparse, y_train_original)):
    print('---', f'fold {index + 1} in folds {stratified_kfolds.n_splits}', '---')

    X_train, y_train = X_train_sparse[train_index], y_train_original[train_index]
    X_valid, y_valid = X_train_sparse[valid_index], y_train_original[valid_index]

    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_valid, y_valid)

    lgb_model = lgb.train(params=parameters,
                          train_set=dtrain,
                          num_boost_round=5000, # 부스팅 반복 횟수를 지정합니다.
                          valid_sets=dvalid, 
                          feval=gini_feval, # 검증 평가 지표로 `gini_feval` 함수를 지정합니다.
                          early_stopping_rounds=100, # 100회 안에 성능이 향상되지 않으면 종료합니다.
                          verbose_eval=50) # 50번째에서 점수를 출력합니다.
    
    # 테스트 세트로 OOF 예측을 수행합니다.
    oof_test_predictions += lgb_model.predict(X_test_sparse)/stratified_kfolds.n_splits
    # 검증 세트로 타깃값을 예측하여 모델 성능을 측정합니다.
    oof_valid_predictions[valid_index] += lgb_model.predict(X_valid)
    
    # 검증 세트로 타깃값을 예측한 배열을 토대로 정규화된 지니계수를 산출합니다.
    normalized_gini_coefficient_final = normalized_gini_coefficient_2(y_valid, oof_valid_predictions[valid_index])
    print(f'fold {index+1}\'s normalized gini coefficient: {normalized_gini_coefficient_final}\n')

--- fold 1 in folds 5 ---


/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1340
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 199
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 100 rounds
[50]	valid_0's binary_logloss: 0.154265	valid_0's gini: 0.246193
[100]	valid_0's binary_logloss: 0.15328	valid_0's gini: 0.258727
[150]	valid_0's binary_logloss: 0.152742	valid_0's gini: 0.265296
[200]	valid_0's binary_logloss: 0.152409	valid_0's gini: 0.269537
[250]	valid_0's binary_logloss: 0.152206	valid_0's gini: 0.272636
[300]	valid_0's binary_logloss: 0.152075	valid_0's gini: 0.274704
[350]	valid_0's binary_logloss: 0.151986	valid_0's gini: 0.276251
[400]	valid_0's binary_logloss: 0.151934	valid_0's gini: 0.27719
[450]	valid_0's binary_logloss: 0.151878	valid_0's gini: 0.278546
[500]	valid_

## **2.5. 성능 측정 및 파일 제출**

In [ ]:
normalized_gini_coefficient_2(y_train_original, oof_valid_predictions)

In [ ]:
submission['target'] = oof_test_predictions
submission.to_csv('Porto Seguro’s Safe Driver Prediction_submission_4.csv', index=False)

프라이빗 스코어는 0.28357입니다. 이전 스코어인 0.28266보다 약간 올랐습니다. 순위는 5156팀 중 2608위에 위치합니다. 이전 순위인 2731위보다 약간 오른 수준입니다.

다음 성능 향상 챕터에서는 파생 피처를 생성해 보겠습니다. 하이퍼파라미터 튜닝은 최종 모델링 챕터에서 수행하겠습니다.